In [1]:
from IrisBackendv3.codec.payload import Payload

In [2]:
## Payload Collection Serialization Testing.
from typing import List

from IrisBackendv3.data_standards import DataStandards
from IrisBackendv3.data_standards.logging import logger as DsLogger
from IrisBackendv3.codec.settings import set_codec_standards

from IrisBackendv3.codec.payload import PayloadCollection, CommandPayload
from IrisBackendv3.codec import Magic

from datetime import datetime
from IrisBackendv3.codec.metadata import UplinkTimes, DownlinkTimes

# Setup some basic Data Standards:
DsLogger.setLevel('CRITICAL')
standards = DataStandards.build_standards()
set_codec_standards(standards)

# Grab a basic NoOp command:
module, command = standards.global_command_lookup('CommandDispatcher_Cmdnoop')
module2, command2 = standards.global_command_lookup('Navigation_NavDriveForward')

# Make a payload containing two of them.
pc = PayloadCollection.make_empty()
pc.CommandPayload.extend([
    CommandPayload(
        magic=Magic.COMMAND,
        module_id=module.ID,
        command_id=command.ID,
        args=dict()
    ),
    CommandPayload(
        magic=Magic.COMMAND,
        module_id=module2.ID,
        command_id=command2.ID,
        args=dict(distance=10, speed=5, callback_id=0x00)
    )
])

In [3]:
from pprint import pformat
print(pformat(pc, indent=4))

PayloadCollection(CommandPayload=[CommandDispatcher_Cmdnoop(0x0800)[], Navigation_NavDriveForward(0x0D00)[distance=10, speed=5, callback_id=0]], TelemetryPayload=[], EventPayload=[], FileBlockPayload=[])


In [4]:
class PayloadTreeReporterList(list):
    """ An extension of `list` used for returning arrays from
    `PayloadTreeNode.payloads` which reports back to the parent
    `PayloadTreeNode` when modified (this way if you, say, add an instance of a
    subclass (CommandPayload) to a node derived from a parent class, that
    change will still end up inside the right node).
    """
    pass # NEVER MIND. This opens too many cans of worms. Easier and better to
    # just deprecate support for such modifications and scrub any such instances
    # from the codebase.

In [6]:
from IrisBackendv3.codec.payload import Payload
from IrisBackendv3.codec.payload_collection import EnhancedPayloadCollection


In [7]:
## Enhanced Payload Collection Serialization Testing.
from typing import List

from IrisBackendv3.data_standards import DataStandards
from IrisBackendv3.data_standards.logging import logger as DsLogger
from IrisBackendv3.codec.settings import set_codec_standards

from IrisBackendv3.codec.payload import CommandPayload
from IrisBackendv3.codec import Magic

from datetime import datetime
from IrisBackendv3.codec.metadata import UplinkTimes, DownlinkTimes

# Setup some basic Data Standards:
DsLogger.setLevel('CRITICAL')
standards = DataStandards.build_standards()
set_codec_standards(standards)

# Grab a basic NoOp command:
module, command = standards.global_command_lookup('CommandDispatcher_Cmdnoop')
module2, command2 = standards.global_command_lookup('Navigation_NavDriveForward')

# Make a payload containing two of them.
epc = EnhancedPayloadCollection(
    UplinkedPayload = [
        CommandPayload(
            magic=Magic.COMMAND,
            module_id=module.ID,
            command_id=command.ID,
            args=dict()
        ),
        CommandPayload(
            magic=Magic.COMMAND,
            module_id=module2.ID,
            command_id=command2.ID,
            args=dict(distance=10, speed=5, callback_id=0x00)
        )
    ]
)

In [8]:
print(str(epc))

{   'CommandPayload': [   CommandDispatcher_Cmdnoop(0x0800)[],
                          Navigation_NavDriveForward(0x0D00)[distance=10, speed=5, callback_id=0]],
    'EventPayload': [],
    'FileBlockPayload': [],
    'TelemetryPayload': [],
    'WatchdogCommandPayload': []}


In [9]:
[*epc]

[[CommandDispatcher_Cmdnoop(0x0800)[],
  Navigation_NavDriveForward(0x0D00)[distance=10, speed=5, callback_id=0]],
 [],
 [],
 [],
 []]

In [15]:
##! TODO (WORKING-HERE):
# `PayloadCollectionInterface` ABC that's used as a meta for both
# `EnhancedPayloadCollection` and `PayloadCollection`.

##!
# !- Note: `chain` will (intentionally) break all .appends and .extends, etc. in the code base using `PayloadCollection`. You'll have to carefully modify every use of `PayloadCollection` to support `EnhancedPayloadCollection`.
# !- Once all this works and `PayloadTreeNode` is no longer a storage class (`.payloads` and `.direct_payloads` are no longer used), remove storage features from `PayloadTreeNode` and retest.
##!

## TODO: Scrub all instances of pc[something].append and .extend from the codebase (this won't actually modify the lists). Instead replace w/


In [10]:
pc = epc

In [11]:
from typing import Iterable
# index by any Payload subclass as . or [int|str|Type[Payload]] or getattr(x, str|Type[Payload]) -> List[Payload]
assert isinstance(pc.CommandPayload, Iterable) and type([*pc.CommandPayload][0]) == CommandPayload
assert isinstance(pc[0], Iterable) and type([*pc[0]][0]) == CommandPayload
assert isinstance(getattr(pc, 'CommandPayload'), Iterable) and type([*getattr(pc, 'CommandPayload')][0]) == CommandPayload
# iterable over all Payload lists -> Iter[List[Payload]]
pc_len = 0
for cs in pc:
    pc_len += 1
    assert isinstance(cs, Iterable)
    if len(cs) > 0:
        assert isinstance(cs[0], Payload)
# __len__ should match Iter[Payload]
assert len(pc) == pc_len

# construct from dict of names
new_pc: EnhancedPayloadCollection = EnhancedPayloadCollection(
    CommandPayload=[],
    TelemetryPayload=[],
    EventPayload=[],
    FileBlockPayload=[]
    #etc.
)

# is pickleable:
import pickle
p = pickle.dumps(pc)
up = pickle.loads(p)
assert up == pc
print(len(p))

# pass by reference (modifying any of the lists that come out of [], etc. should modify the data)
# -- re: checking what's put in there: I'll make sure I always put stuff in the right place but I don't care if you do
# -- Should make append(Payload) and extend(PayloadCollection|List[Payload]) function to do this correctly.
n_payloads = epc.num_payloads
x = [*pc[CommandPayload]][1]
x._args['distance'] = 0xBE
assert x == [*pc[CommandPayload]][1]
assert x == [*pc.CommandPayload][1]

epc.append(
        CommandPayload(
            magic=Magic.COMMAND,
            module_id=module.ID,
            command_id=command.ID,
            args=dict()
        )
)
assert epc.num_payloads == n_payloads+1

# make sure all are of the right type when setting but otherwise just feed in
# needs to be fundamentally encodeable, like a namedtuple (in its core dataset)
# should be able to duck-construct it from any Dict[str, List[Payload]|Dict[Type[Payload], List[Payload]] or List[Tuple[str, List[Payload]]]|List[Tuple[Type[Payload], List[Payload]]


674


In [12]:
546

546

In [13]:
import IrisBackendv3.ipc as ipc
import IrisBackendv3.ipc.restricted_pickler as rp

In [14]:
from datetime import datetime
from IrisBackendv3.codec.metadata import UplinkTimes, DownlinkTimes
import pickle

dt1 = datetime.now()
bdt = rp.restricted_dumps(dt1)
dt2 = rp.restricted_loads(bdt)
assert dt2 == dt1

dt = DownlinkTimes(pmcc_rx = datetime.now())
ut1 = UplinkTimes(generated = datetime.now())
ut1.ack_downlink_times = dt
but1 = rp.restricted_dumps(ut1)
ut2 = rp.restricted_loads(but1)
assert ut1 == ut2
ut2

UplinkTimes(generated=datetime.datetime(2022, 5, 6, 16, 36, 58, 19100), pmcc_rx=None, pmcc_tx=None, amcc_rx=None, ack_rover=None, ack_downlink_times=DownlinkTimes(lander_rx=None, amcc_rx=None, pmcc_rx=datetime.datetime(2022, 5, 6, 16, 36, 58, 19061)))

In [5]:
from trans_tools import *
from __command_aliases import get_command

seq_num = 0x01
pathway, magic, command_name, kwargs, telem_pathway = get_command('SetBatteryConnection', True)
packet1 = build_command_packet(seq_num, pathway, magic, command_name, kwargs)

bpacket1 = rp.restricted_dumps(packet1)
packet2 = rp.restricted_loads(bpacket1)
packet1, packet2

(ICP[#1::7]: 	0 T	- 0 E	- 0 B	- 1 C, ICP[#1::7]: 	0 T	- 0 E	- 0 B	- 1 C)

In [9]:
## Payload Collection Serialization Testing.
from typing import List, ClassVar, Set

from IrisBackendv3.data_standards import DataStandards
from IrisBackendv3.data_standards.logging import logger as DsLogger
from IrisBackendv3.codec.settings import set_codec_standards

from IrisBackendv3.codec.payload import PayloadCollection, CommandPayload

from datetime import datetime
from IrisBackendv3.codec.metadata import UplinkTimes, DownlinkTimes

# Setup some basic Data Standards:
DsLogger.setLevel('CRITICAL')
standards = DataStandards.build_standards()
set_codec_standards(standards)

# Grab a basic NoOp command:
module, command = standards.global_command_lookup('CommandDispatcher_Cmdnoop')

# Make a payload containing two of them.
pc = PayloadCollection.make_empty()
pc.CommandPayload.extend([
    CommandPayload(
        magic=Magic.COMMAND,
        module_id=module.ID,
        command_id=command.ID,
        args=dict()
    ),
    CommandPayload(
        magic=Magic.COMMAND,
        module_id=module.ID,
        command_id=command.ID,
        args=dict()
    )
])

import pickle
decoder, data, state = pc.CommandPayload[0].__reduce__()
print((data,state))
b = pickle.dumps(pc)

((b'\x00\x08', '<'), {'magic': <Magic.COMMAND: 12245589>, 'pathway': <DataPathway.NONE: -1>, 'source': <DataSource.NONE: -1>, 'amcc_ack': False, 'rover_ack': False, 'uplink_times': UplinkTimes(generated=datetime.datetime(2022, 3, 28, 23, 0, 57, 438202), pmcc_rx=None, pmcc_tx=None, amcc_rx=None, ack_rover=None, ack_downlink_times=DownlinkTimes(lander_rx=None, amcc_rx=None, pmcc_rx=None))})


In [10]:
from IrisBackendv3.ipc.restricted_pickler import restricted_dumps, restricted_loads
from IrisBackendv3.codec.packet import IrisCommonPacket
icp = IrisCommonPacket(seq_num = 0, payloads = pc)

smsg = restricted_dumps(icp)
assert icp == restricted_loads(smsg)
smsg, len(smsg)

(b'e6ad020794e23258251a9c57b3cbef80c45433f06121580baebbeb402d62f09f 4aaf06ad901be4bd19a1ba1d72745e5a809e3660362675c642f9c4e0789618a9 \x80\x04\x95\xc0\x02\x00\x00\x00\x00\x00\x00\x8c\x08builtins\x94\x8c\x07getattr\x94\x93\x94\x8c\x1aIrisBackendv3.codec.packet\x94\x8c\x10IrisCommonPacket\x94\x93\x94\x8c\x14build_minimum_packet\x94\x86\x94R\x94\x8c\x1bIrisBackendv3.codec.payload\x94\x8c\x11PayloadCollection\x94\x93\x94(]\x94(h\x02h\t\x8c\x0eCommandPayload\x94\x93\x94\x8c\x06decode\x94\x86\x94R\x94C\x02\x00\x08\x94\x8c\x01<\x94\x86\x94R\x94}\x94(\x8c\x05magic\x94\x8c\x19IrisBackendv3.codec.magic\x94\x8c\x05Magic\x94\x93\x94JU\xda\xba\x00\x85\x94R\x94\x8c\x07pathway\x94\x8c\x1cIrisBackendv3.codec.metadata\x94\x8c\x0bDataPathway\x94\x93\x94J\xff\xff\xff\xff\x85\x94R\x94\x8c\x06source\x94h\x1e\x8c\nDataSource\x94\x93\x94J\xff\xff\xff\xff\x85\x94R\x94\x8c\x08amcc_ack\x94\x89\x8c\trover_ack\x94\x89\x8c\x0cuplink_times\x94h\x1e\x8c\x0bUplinkTimes\x94\x93\x94)\x81\x94(\x8c\x08datetime\x94\x8c\x08

In [17]:
import zlib
cmsg = zlib.compress(smsg, level=1)
assert icp == restricted_loads(zlib.decompress(cmsg))
cmsg, len(cmsg)

(b'x\x01mR\xcbn\xd3@\x145I\x9b\xa4\x0fJ\xab"U@@\x15]\x90"\x14\x8d\xdf\xb6\xc4\x8at\x01H\xa0\x08\xe8\xb2\x8a\xe6\xe5\x8e\xd5\xd8\x8e\x12\xa7U\x17Ht\x01U\xa4\xd9\xe5\xf2\x1d\xec\xbb\xe2\'\xd8\xf0\x01\xfc\x06\xcc8\x16X(\xd7\x92}\xef\xdc3\xe7\x1e\x1f]\xeea\x86,\xe4\x87\x0e\xb7l\xcb\r,\xd7\xc4!u}bS\xc2\xa3\x00Q\xc7ul;B\x9ei\x99n\x80\x08\xe6\x84p\xe2 \x8byV\x84\xc2h\xdf\xc1Xu1\x0b\x91I\xb8C\x98\x19b\x93`\x93\xf9\x96\xef\xb8\xdc\xc5\x01\n\xb9\xedy\xc8\xf6,\xcfw\xa9\xe7XQH\x1d\x8e\xfc \xf4\xcc\x00\x87\xfb\x9fV\xbe~\xaf\x19E\xc8\x16\x99\xc6\xc3<N\' \x9b\xa7<\xc7y>\x869\xc8\xfb\xaf\xc6\xf1\xe4\x05\xa6g<e\xe7v\x97f\x8c\xd3\xeeH\xd79\xc8m\xdd\xeceI\x92\xa5\xfd\xc5\x91\xbaqW\x13\xb1A\x12\xa7q2M\x06%\xf6\x0b\xbc\x03\xf9`9\xd9\xe50\xc3\x0c\xe4N\x1f\x17Y/\x1b\x0e9\xcd\xe3,U\x02:\'\xd0\x115\xb1&\xb7\xf4\x1c\x9c\xb2\x12\xa4\xb55\x94\x16\xa5\x074w\xaff\xb4@\xdez^\x14\x1f\xa1#W\x13|\x1aS\x90\xf7\x96\xcd,{\xabo\n\xcc\x1c^\x1f\xff\xbc1>k\x89\xcd\x11\xce\xc5\x05\xbe\x04\xd9^zQ9\xc3p\x8eAn\x1c\xa9O\xbf\x04+\x

In [2]:
context = ipc.create_context()
pub_socket = ipc.create_socket(
    context,
    socket_type=ipc.SocketType.PUBLISHER,
    ports=ipc.Port.TRANSCEIVER
)

2022-02-26 17:14:12 connors-mbp.wifi.local.cmu.edu IrisBackendv3.ipc.logging[44160] INFO Created a `SocketType.PUBLISHER` at `tcp://127.0.0.1:61453` (port Port.TRANSCEIVER: F00D).


In [2]:
## SETUP:
from trans_tools import *
from IrisBackendv3.codec.payload import CommandPayload
from IrisBackendv3.codec.packet import IrisCommonPacket
from IrisBackendv3.codec.metadata import DataPathway, DataSource
from IrisBackendv3.codec.magic import Magic
from IrisBackendv3.utils.basic import bytearray_to_spaced_hex as hexstr
from scapy.utils import hexdump

wp = b'\xf4\x01\xfd\x0c<\x004!\x10\x00\x01\x00\x00'

bad_herc_packets = [
    b'\x7f\x0b\x00\x00\xa5\xfb\xff\x10\x00\xc0\x04\x0f\x7f\x0b\x00\x00\x9a\xf7\xff\x10\x00\xc0\x05\x0f\x7f\x0b\x00\x00\x15\xfb\xff\x10\x00\xc0\x00\x0fZ\x10\x00\x00\xf7\xff\xff\x10\x00\xc0\x01\x0fZ\x10\x00\x00\xf7\xff\xff\x10\x00\xc0\xe0\x00\xce\x03\x00\x00\x0b\xb0!\x1f\x00\x00\x04\x00\xff\x0f\x00\x00\x00\x00\x00\x00\xff\x01\x00\x00\x00\x00\x00\x00\x0b\xb0!#\x00\x00\x00\x00\xff\x0f\x00\x00\x00\x00\x00\x00\xff\x01\x00\x00\x00\x00\x00\x00\x0b\xb0!#\x00\x00\x00\x00\xff\x0f\x00\x00\x00\x00\x00\x00\xff\x01\x00\x00\x00\x00\x00\x00\x0b\xb0!#\x00\x00\x00\x00\xff\x0f\x00\x00\x00\x00\x00\x00\xff\x01\x00\x00\x00\x00\x00\x00\x0b\xb0!#\x00\x00\x00\x00\xff\x0f\x00\x00\x00\x00\x00\x00\xff\x01\x00\x00\x00\x00\x00\x00\x0b\xb0!#\x00\x00\x00\x00\xff\x0f\x00\x00\x00\x00\x00\x00\xff\x01\x00\x00\x00\x00\x00\x00\x0b\xb0!#\x00\x00\x00\x00\xff\x0f\x00\x00\x00\x00\x00\x00\xff\x01\x00\x00\x00\x00\x00\x00\x0b\xb0!#\x00\x00\x00\x00\xff\x0f\x00\x00\x00\x00\x00\x00\xff\x01\x00\x00\x00\x00\x00\x00\x0b\xb0!#\x00\x00\x00\x00\xff\x0f\x00\x00\x00\x00\x00\x00\xff\x01\x00\x00\x00\x00\x00\x00\x0b\xb0',
    b'\xc0\x05\x0fy/\x00\x00\x15\xfb\xff\x10\x00\xc0\x00\x0fl0\x00\x00\xfa\xff\xff\x10\x00\xc0\x01\x0fl0\x00\x00\xfe\xff\xff\x10\x00\xc0\x02\x0fl0\x00\x00\xa8\x00\xff\x10\x00\xc0\x03\x0fl0\x00\x00\xa5\xfb\xe4\x00\xd2\x03\x00\x00\x0b\xb0!\x1f\x00\x00\x04\x00\xff\x0f\x00\x00\x00\x00\x00\x00\xff\x01\x00\x00\x00\x00\x00\x00\x0b\xb0!#\x00\x00\x00\x00\xff\x0f\x00\x00\x00\x00\x00\x00\xff\x01\x00\x00\x00\x00\x00\x00\x0b\xb0!#\x00\x00\x00\x00\xff\x0f\x00\x00\x00\x00\x00\x00\xff\x01\x00\x00\x00\x00\x00\x00\x0b\xb0!#\x00\x00\x00\x00\xff\x0f\x00\x00\x00\x00\x00\x00\xff\x01\x00\x00\x00\x00\x00\x00\x0b\xb0!#\x00\x00\x00\x00\xff\x0f\x00\x00\x00\x00\x00\x00\xff\x01\x00\x00\x00\x00\x00\x00\x0b\xb0!#\x00\x00\x00\x00\xff\x0f\x00\x00\x00\x00\x00\x00\xff\x01\x00\x00\x00\x00\x00\x00\x0b\xb0!#\x00\x00\x00\x00\xff\x0f\x00\x00\x00\x00\x00\x00\xff\x01\x00\x00\x00\x00\x00\x00\x0b\xb0!#\x00\x00\x00\x00\xff\x0f\x00\x00\x00\x00\x00\x00\xff\x01\x00\x00\x00\x00\x00\x00\x0b\xb0!#\x00\x00\x00\x00\xff\x0f\x00\x00\x00\x00\x00\x00\xff\x01\x00\x00\x00\x00\x00\x00\x0b\xb0!#\x00\x00',
    b'\xc0\x01\x0f\x83P\x00\x00\xf8\xff\xff\x10\x00\xc0\x02\x0f\x83P\x00\x00\xa7\x00\xff\x10\x00\xc0\x03\x0f\x83P\x00\x00\xa5\xfb\xff\x10\x00\xc0\x04\x0f\x83P\x00\x00\x9a\xf7\xff\x10\x00\xc0\x05\x0f\x83P\x00\x00\x15\xfb\xe4\x00\xd2\x03\x00\x00\x00\x00\xff\x0f\x00\x00\x00\x00\x00\x00\xff\x01\x00\x00\x00\x00\x00\x00\x0b\xb0!#\x00\x00\x00\x00\xff\x0f\x00\x00\x00\x00\x00\x00\xff\x01\x00\x00\x00\x00\x00\x00\x0b\xb0!#\x00\x00\x00\x00\xff\x0f\x00\x00\x00\x00\x00\x00\xff\x01\x00\x00\x00\x00\x00\x00\x0b\xb0!#\x00\x00\x00\x00\xff\x0f\x00\x00\x00\x00\x00\x00\xff\x01\x00\x00\x00\x00\x00\x00\x0b\xb0!#\x00\x00\x00\x00\xff\x0f\x00\x00\x00\x00\x00\x00\xff\x01\x00\x00\x00\x00\x00\x00\x0b\xb0!#\x00\x00\x00\x00\xff\x0f\x00\x00\x00\x00\x00\x00\xff\x01\x00\x00\x00\x00\x00\x00\x0b\xb0!#\x00\x00\x00\x00\xff\x0f\x00\x00\x00\x00\x00\x00\xff\x01\x00\x00\x00\x00\x00\x00\x0b\xb0!#\x00\x00\x00\x00\xff\x0f\x00\x00\x00\x00\x00\x00\xff\x01\x00\x00\x00\x00\x00\x00\x0b\xb0!#\x00\x00\x00\x00\xff\x0f\x00\x00\x00\x00\x00\x00\xff\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x0b\xb0!#\x00\x00',
    b'\x04\x0f\xafp\x00\x00\x9a\xf7\xff\x10\x00\xc0\x05\x0f\xb0p\x00\x00\x15\xfb\xff\x10\x00\xc0\x00\x0f\x8au\x00\x00\xf9\xff\xff\x10\x00\xc0\x01\x0f\x8au\x00\x00\xf8\xff\xff\x10\x00\xc0\x02\x0f\x8au\x00\x00\xae\x00\xff\x10\xe2\x00\xd0\x03\x00\x00\x00\x00\xff\x0f\x00\x00\x00\x00\x00\x00\xff\x01\x00\x00\x00\x00\x00\x00\x0b\xb0!#\x00\x00\x00\x00\xff\x0f\x00\x00\x00\x00\x00\x00\xff\x01\x00\x00\x00\x00\x00\x00\x0b\xb0!#\x00\x00\x00\x00\xff\x0f\x00\x00\x00\x00\x00\x00\xff\x01\x00\x00\x00\x00\x00\x00\x0b\xb0!#\x00\x00\x00\x00\xff\x0f\x00\x00\x00\x00\x00\x00\xff\x01\x00\x00\x00\x00\x00\x00\x0b\xb0!#\x00\x00\x00\x00\xff\x0f\x00\x00\x00\x00\x00\x00\xff\x01\x00\x00\x00\x00\x00\x00\x0b\xb0!#\x00\x00\x00\x00\xff\x0f\x00\x00\x00\x00\x00\x00\xff\x01\x00\x00\x00\x00\x00\x00\x0b\xb0!#\x00\x00\x00\x00\xff\x0f\x00\x00\x00\x00\x00\x00\xff\x01\x00\x00\x00\x00\x00\x00\x0b\xb0!#\x00\x00\x00\x00\xff\x0f\x00\x00\x00\x00\x00\x00\xff\x01\x00\x00\x00\x00\x00\x00\x0b\xb0!#\x00\x00\x00\x00\xff\x0f\x00\x00\x00\x00\x00\x00\xff\x01\x00\x00\x00\x00\x00\x00\x0b\xb0!#\x00\x00\x00\x00\xff\x0f',
    b'\x04\x0f:\xb7\x00\x00\x9a\xf7\xff\x10\x00\xc0\x05\x0f:\xb7\x00\x00\x15\xfb\xff\x10\x00\xc0\x00\x0f\x15\xbc\x00\x00\xf8\xff\xff\x10\x00\xc0\x01\x0f\x15\xbc\x00\x00\xf7\xff\xff\x10\x00\xc0\x02\x0f\x15\xbc\x00\x00\xad\x00\xff\x10\xe2\x00\xd0\x03\x00\x00\x00\x00\x00\x00\xff\x01\x00\x00\x00\x00\x00\x00\x0b\xb0!#\x00\x00\x00\x00\xff\x0f\x00\x00\x00\x00\x00\x00\xff\x01\x00\x00\x00\x00\x00\x00\x0b\xb0!#\x00\x00\x00\x00\xff\x0f\x00\x00\x00\x00\x00\x00\xff\x01\x00\x00\x00\x00\x00\x00\x0b\xb0!#\x00\x00\x00\x00\xff\x0f\x00\x00\x00\x00\x00\x00\xff\x01\x00\x00\x00\x00\x00\x00\x0b\xb0!#\x00\x00\x00\x00\xff\x0f\x00\x00\x00\x00\x00\x00\xff\x01\x00\x00\x00\x00\x00\x00\x0b\xb0!#\x00\x00\x00\x00\xff\x0f\x00\x00\x00\x00\x00\x00\xff\x01\x00\x00\x00\x00\x00\x00\x0b\xb0!#\x00\x00\x00\x00\xff\x0f\x00\x00\x00\x00\x00\x00\xff\x01\x00\x00\x00\x00\x00\x00\x0b\xb0!#\x00\x00\x00\x00\xff\x0f\x00\x00\x00\x00\x00\x00\xff\x01\x00\x00\x00\x00\x00\x00\x0b\xb0!#\x00\x00\x00\x00\xff\x0f\x00\x00\x00\x00\x00\x00\xff\x01\x00\x00\x00\x00\x00\x00\x0b\xb0!#\x00\x00\x00\x00\xff\x0f\x00\x00\x00\x00\x00\x00',
    b'\x04\x0f\x92\x1c\n\x00\x9a\xf7\xff\x10\x00\xc0\x05\x0f\x92\x1c\n\x00\x15\xfb\xff\x10\x00\xc0\x00\x0f\x85\x1d\n\x00\xf8\xff\xff\x10\x00\xc0\x01\x0f\x85\x1d\n\x00\xf8\xff\xff\x10\x00\xc0\x02\x0f\x85\x1d\n\x00\xac\x00\xff\x10\xe2\x00\xd0\x03\x00\x00\x0b\xb0!#\x00\x00\x00\x00\xff\x0f\x00\x00\x00\x00\x00\x00\xff\x01\x00\x00\x00\x00\x00\x00\x0b\xb0!#\x00\x00\x00\x00\xff\x0f\x00\x00\x00\x00\x00\x00\xff\x01\x00\x00\x00\x00\x00\x00\x0b\xb0!#\x00\x00\x00\x00\xff\x0f\x00\x00\x00\x00\x00\x00\xff\x01\x00\x00\x00\x00\x00\x00\x0b\xb0!#\x00\x00\x00\x00\xff\x0f\x00\x00\x00\x00\x00\x00\xff\x01\x00\x00\x00\x00\x00\x00\x0b\xb0!#\x00\x00\x00\x00\xff\x0f\x00\x00\x00\x00\x00\x00\xff\x01\x00\x00\x00\x00\x00\x00\x0b\xb0!#\x00\x00\x00\x00\xff\x0f\x00\x00\x00\x00\x00\x00\xff\x01\x00\x00\x00\x00\x00\x00\x0b\xb0!#\x00\x00\x00\x00\xff\x0f\x00\x00\x00\x00\x00\x00\xff\x01\x00\x00\x00\x00\x00\x00\x0b\xb0!#\x00\x00\x00\x00\xff\x0f\x00\x00\x00\x00\x00\x00\xff\x01\x00\x00\x00\x00\x00\x00\x0b\xb0!#\x00\x00\x00\x00\xff\x0f\x00\x00\x00\x00\x00\x00\xff\x01\x00\x00\x00\x00\x00\x00\x0b\xb0!#',
    b'\x04\x0f\xbb\xe7\n\x00\x9a\xf7\xff\x10\x00\xc0\x05\x0f\xbb\xe7\n\x00\x15\xfb\xff\x10\x00\xc0\x00\x0f\xad\xe8\n\x00\xf8\xff\xff\x10\x00\xc0\x01\x0f\xae\xe8\n\x00\xf9\xff\xff\x10\x00\xc0\x02\x0f\xae\xe8\n\x00\xaa\x00\xff\x10\xe2\x00\xd0\x03\x00\x00\x00\x00\x00\x00\x00\x00\x0b\xb0!#\x00\x00\x00\x00\xff\x0f\x00\x00\x00\x00\x00\x00\xff\x01\x00\x00\x00\x00\x00\x00\x0b\xb0!#\x00\x00\x00\x00\xff\x0f\x00\x00\x00\x00\x00\x00\xff\x01\x00\x00\x00\x00\x00\x00\x0b\xb0!#\x00\x00\x00\x00\xff\x0f\x00\x00\x00\x00\x00\x00\xff\x01\x00\x00\x00\x00\x00\x00\x0b\xb0!#\x00\x00\x00\x00\xff\x0f\x00\x00\x00\x00\x00\x00\xff\x01\x00\x00\x00\x00\x00\x00\x0b\xb0!#\x00\x00\x00\x00\xff\x0f\x00\x00\x00\x00\x00\x00\xff\x01\x00\x00\x00\x00\x00\x00\x0b\xb0!#\x00\x00\x00\x00\xff\x0f\x00\x00\x00\x00\x00\x00\xff\x01\x00\x00\x00\x00\x00\x00\x0b\xb0!#\x00\x00\x00\x00\xff\x0f\x00\x00\x00\x00\x00\x00\xff\x01\x00\x00\x00\x00\x00\x00\x0b\xb0!#\x00\x00\x00\x00\xff\x0f\x00\x00\x00\x00\x00\x00\xff\x01\x00\x00\x00\x00\x00\x00\x0b\xb0!#\x00\x00\x00\x00\xff\x0f\x00\x00\x00\x00\x00\x00\xff\x01\x00\x00\x00\x00'
]
bhp = bad_herc_packets

bhp2 = [
    b'\xff\x01\x00\x00\x00\x00\x00\x00\x0b\xb0!#',
    b'\x03\x0f\xe9uB\x00\xa5\xfb\xff\x10\x00\xc0\x04\x0f\xe9uB\x00\x9a\xf7\xff\x10\x00\xc0\x05\x0f\xe9uB\x00\x15\xfb\xff\x10\x00\xc0\x00\x0f\xdcvB\x00\x01\x00\xff\x10\x00\xc0\x01\x0f\xdcvB\x00\x01\x00\xff\x10\xe2\x00\xd0\x03\x00\x00\x00\x00\xff\x0f\x00\x00\x00\x00\x00\x00\xff\x01\x00\x00\x00\x00\x00\x00\x0b\xb0!#\x00\x00\x00\x00\xff\x0f\x00\x00\x00\x00\x00\x00\xff\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x0b\xb0!#\x00\x00\x00\x00\xff\x0f\x00\x00\x00\x00\x00\x00\xff\x01\x00\x00\x00\x00\x00\x00\x0b\xb0!#\x00\x00\x00\x00\xff\x0f\x00\x00\x00\x00\x00\x00\xff\x01\x00\x00\x00\x00\x00\x00\x0b\xb0!#\x00\x00\x00\x00\xff\x0f\x00\x00\x00\x00\x00\x00\xff\x01\x00\x00\x00\x00\x00\x00\x0b\xb0!#\x00\x00\x00\x00\xff\x0f\x00\x00\x00\x00\x00\x00\xff\x01\x00\x00\x00\x00\x00\x00\x0b\xb0!#\x00\x00\x00\x00\xff\x0f\x00\x00\x00\x00\x00\x00\xff\x01\x00\x00\x00\x00\x00\x00\x0b\xb0!#\x00\x00\x00\x00\xff\x0f\x00\x00\x00\x00\x00\x00\xff\x01\x00\x00\x00\x00\x00\x00\x0b\xb0!#\x00\x00\x00\x00\xff\x0f\x00\x00\x00\x00\x00\x00\xff\x01\x00\x00\x00\x00\x00\x00\x0b\xb0!#',
    b'gB\x00\x15\xfb\xff\x10\x00\xc0\x00\x0f\xb1lB\x00\xf4\xff\xff\x10\x00\xc0\x01\x0f\xb1',
    b'\xfb\xff\x10\x00\xc0\x00\x0f\xf2\x96B\x00\xf0\xff\xff\x10\x00\xc0\x01\x0f\xf2\x96B\x00\xf4\xff\xff\x10\x00\xc0\x02\x0f\xf2\x96B\x00\xa5\x00\xff\x10\x00\xc0\x03\x0f\xf2\x96B\x00\xa5\xfb\xff\x10\x00\xc0\x04\x0f\xf2\x96B\x00\x9a\xf7\xff\x10\xe2\x00\xd0\x03\x00\x00\x00\x00\xff\x0f\x00\x00\x00\x00\x00\x00\xff\x01\x00\x00\x00\x00\x00\x00\x0b\xb0!#\x00\x00\x00\x00\xff\x0f\x00\x00\x00\x00\x00\x00\xff\x01\x00\x00\x00\x00\x00\x00\x0b\xb0!#\x00\x00\x00\x00\xff\x0f\x00\x00\x00\x00\x00\x00\xff\x01\x00\x00\x00\x00\x00\x00\x0b\xb0!#\x00\x00\x00\x00\xff\x0f\x00\x00\x00\x00\x00\x00\xff\x01\x00\x00\x00\x00\x00\x00\x0b\xb0!#\x00\x00\x00\x00\xff\x0f\x00\x00\x00\x00\x00\x00\xff\x01\x00\x00\x00\x00\x00\x00\x0b\xb0!#\x00\x00\x00\x00\xff\x0f\x00\x00\x00\x00\x00\x00\xff\x01\x00\x00\x00\x00\x00\x00\x0b\xb0!#\x00\x00\x00\x00\xff\x0f\x00\x00\x00\x00\x00\x00\xff\x01\x00\x00\x00\x00\x00\x00\x0b\xb0!#\x00\x00\x00\x00\xff\x0f\x00\x00\x00\x00\x00\x00\xff\x01\x00\x00\x00\x00\x00\x00\x0b\xb0!#\x00\x00\x00\x00\xff\x0f\x00\x00\x00\x00\x00\x00\xff\x01\x00\x00\x00\x00\x00\x00\x0b\xb0!#\x00\x00\x00\x00\xff\x0f',
    b'\xfb\xff\x10\x00\xc0\x00\x0fJ\x8eB\x00\x00\x00\xff\x10\x00\xc0\x01\x0fJ\x8eB\x00\xfe\xff\xff\x10\x00\xc0\x02\x0fJ\x8eB\x00\xaf\x00\xff\x10\x00\xc0\x03\x0fJ\x8eB\x00\xa5\xfb\xff\x10\x00\xc0\x04\x0fJ\x8eB\x00\x9a\xf7\xff\x10\x00\xc0\x05\x0fJ\x8eB\x00\x15\xfb\xff\x10\x00\xc0\x00\x0f%\x93B\x00\xf7\xff\xff\x10\x00\xc0\x01\x0f%\x93B\x00\xfb\xff\xff\x10\x00\xc0\x02\x0f%\x93B\x00\xaa\x00\xff\x10\x00\xc0\x03\x0f%\x93B\x00\xa5\xfb\xff\x10\x00\xc0\x04\x0f%\x93B\x00\x9a\xf7\xff\x10\x00\xc0\x05\x0f%\x93B\x00\x15\xfb\xff\x10\x00\xc0\x00\x0f\x18\x94B\x00\xf3\xff\xff\x10\x00\xc0\x01\x0f\x18\x94B\x00\xf8\xff\xff\x10\x00\xc0\x02\x0f\x18\x94B\x00\xa7\x00\xff\x10\x00\xc0\x03\x0f\x18\x94B\x00\xa5\xfb\xff\x10\x00\xc0\x04\x0f\x19\x94B\x00\x9a\xf7\xff\x10\x00\xc0\x05\x0f\x19\x94B\x00\x15\xfb\xff\x10\x00\xc0\x00\x0f\x0c\x95B\x00\xf9\xff\xff\x10\x00\xc0\x01\x0f\x0c\x95B\x00\xfc\xff\xff\x10\x00\xc0\x02\x0f\x0c\x95B\x00\xac\x00\xff\x10\x00\xc0\x03\x0f\x0c\x95B\x00\xa5\xfb\xff\x10\x00\xc0\x04\x0f\x0c\x95B\x00\x9a\xf7\xff\x10\x00\xc0\x05\x0f\x0c\x95B\x00\x15\xfb\xff\x10\x00\xc0\x00\x0f\xff\x95B\x00\xf8\xff\xff\x10\x00\xc0\x01\x0f\xff\x95B\x00\x00\x00\xff\x10\x00\xc0\x02\x0f\xff\x95B\x00\xb1\x00\xff\x10\x00\xc0\x03\x0f\xff\x95B\x00\xa5\xfb\xff\x10\x00\xc0\x04\x0f\xff\x95B\x00\x9a\xf7\xff\x10\x00\xc0\x05\x0f\xff\x95B\x00\x15\xfb\xff\x10\x00\xc0\x00\x0f\xf2\x96B\x00\xf0\xff\xff\x10\x00\xc0\x01\x0f\xf2\x96B\x00\xf4\xff\xff\x10\x00\xc0\x02\x0f\xf2\x96B\x00\xa5\x00\xff\x10\x00\xc0\x03\x0f\xf2\x96B\x00\xa5\xfb\xff\x10\x00\xc0\x04\x0f\xf2\x96B\x00\x9a\xf7\xff\x10\xe2\x00\xd0\x03\x00\x00\x00\x00\xff\x0f\x00\x00\x00\x00\x00\x00\xff\x01\x00\x00\x00\x00\x00\x00\x0b\xb0!#\x00\x00\x00\x00\xff\x0f\x00\x00\x00\x00\x00\x00\xff\x01\x00\x00\x00\x00\x00\x00\x0b\xb0!#\x00\x00\x00\x00\xff\x0f\x00\x00\x00\x00\x00\x00\xff\x01\x00\x00\x00\x00\x00\x00\x0b\xb0!#\x00\x00\x00\x00\xff\x0f\x00\x00\x00\x00\x00\x00\xff\x01\x00\x00\x00\x00\x00\x00\x0b\xb0!#\x00\x00\x00\x00\xff\x0f\x00\x00\x00\x00\x00\x00\xff\x01\x00\x00\x00\x00\x00\x00\x0b\xb0!#\x00\x00\x00\x00\xff\x0f\x00\x00\x00\x00\x00\x00\xff\x01\x00\x00\x00\x00\x00\x00\x0b\xb0!#\x00\x00\x00\x00\xff\x0f\x00\x00\x00\x00\x00\x00\xff\x01\x00\x00\x00\x00\x00\x00\x0b\xb0!#\x00\x00\x00\x00\xff\x0f\x00\x00\x00\x00\x00\x00\xff\x01\x00\x00\x00\x00\x00\x00\x0b\xb0!#\x00\x00\x00\x00\xff\x0f\x00\x00\x00\x00\x00\x00\xff\x01\x00\x00\x00\x00\x00\x00\x0b\xb0!#\x00\x00\x00\x00\xff\x0f'
]

bhp4 = [
    b'\x7f\x0b\x00\x00\xa5\xfb\xff\x10\x00\xc0\x04\x0f\x7f\x0b\x00\x00\x9a\xf7\xff\x10\x00\xc0\x05\x0f\x7f\x0b\x00\x00\x15\xfb\xff\x10\x00\xc0\x00\x0fZ\x10\x00\x00\xfb\xff\xff\x10\x00\xc0\x01\x0fZ\x10\x00\x00\xf6\xff\xff\x10\x00\xc0\xe0\x00\xce\x03\x00\x00\x0b\xb0!#\x00\x00\x00\x00\xff\x0f\x00\x00\x00\x00\x00\x00\xff\x01\x00\x00\x00\x00\x00\x00\x0b\xb0!#\x00\x00\x00\x00\xff\x0f\x00\x00\x00\x00\x00\x00\xff\x01\x00\x00\x00\x00\x00\x00\x0b\xb0!#\x00\x00\x00\x00\xff\x0f\x00\x00\x00\x00\x00\x00\xff\x01\x00\x00\x00\x00\x00\x00\x0b\xb0!#\x00\x00\x00\x00\xff\x0f\x00\x00\x00\x00\x00\x00\xff\x01\x00\x00\x00\x00\x00\x00\x0b\xb0!#\x00\x00\x00\x00\xff\x0f\x00\x00\x00\x00\x00\x00\xff\x01\x00\x00\x00\x00\x00\x00\x0b\xb0!#\x00\x00\x00\x00\xff\x0f\x00\x00\x00\x00\x00\x00\xff\x01\x00\x00\x00\x00\x00\x00\x0b\xb0!#\x00\x00\x00\x00\xff\x0f\x00\x00\x00\x00\x00\x00\xff\x01\x00\x00\x00\x00\x00\x00\x0b\xb0!#\x00\x00\x00\x00\xff\x0f\x00\x00\x00\x00\x00\x00\xff\x01\x00\x00\x00\x00\x00\x00\x0b\xb0!#\x00\x00\x00\x00\xff\x0f\x00\x00\x00\x00\x00\x00\xff\x01\x00\x00\x00\x00\x00\x00\x0b\xb0',
    b'\x00\x00\xff\x0f\x00\x00\x00\x00\x00\x00\xff\x01\x00\x00\x00\x00\x00\x00\x0b\xb0',

]
weird_solo_packet_frag__during_wifi_connect_fail = b'\xff\xff\xff\x0f\x00\x00\x00\x00\x00\x00\x00\x00\xf4\x01\xfd\x0c<\x004!\x10\x00\x01\x00\x00'

```
[282, 285, 285, 284, 284, 284, 284]
0000  04 0F AF 70 00 00 9A F7 FF 10 00 C0 05 0F B0 70  ...p...........p
0010  00 00 15 FB FF 10 00 C0 00 0F 8A 75 00 00 F9 FF  ...........u....
0020  FF 10 00 C0 01 0F 8A 75 00 00 F8 FF FF 10 00 C0  .......u........
0030  02 0F 8A 75 00 00 AE 00 FF 10 E2 00 D0 03 00 00  ...u............
0040  00 00 FF 0F 00 00 00 00 00 00 FF 01 00 00 00 00  ................
0050  00 00 0B B0 21 23 00 00 00 00 FF 0F 00 00 00 00  ....!#..........
0060  00 00 FF 01 00 00 00 00 00 00 0B B0 21 23 00 00  ............!#..
0070  00 00 FF 0F 00 00 00 00 00 00 FF 01 00 00 00 00  ................
0080  00 00 0B B0 21 23 00 00 00 00 FF 0F 00 00 00 00  ....!#..........
0090  00 00 FF 01 00 00 00 00 00 00 0B B0 21 23 00 00  ............!#..
00a0  00 00 FF 0F 00 00 00 00 00 00 FF 01 00 00 00 00  ................
00b0  00 00 0B B0 21 23 00 00 00 00 FF 0F 00 00 00 00  ....!#..........
00c0  00 00 FF 01 00 00 00 00 00 00 0B B0 21 23 00 00  ............!#..
00d0  00 00 FF 0F 00 00 00 00 00 00 FF 01 00 00 00 00  ................
00e0  00 00 0B B0 21 23 00 00 00 00 FF 0F 00 00 00 00  ....!#..........
00f0  00 00 FF 01 00 00 00 00 00 00 0B B0 21 23 00 00  ............!#..
0100  00 00 FF 0F 00 00 00 00 00 00 FF 01 00 00 00 00  ................
0110  00 00 0B B0 21 23 00 00 00 00 FF 0F              ....!#......
----
'0x3d200e4'
```

In [6]:
# print(f"{enb!r}")

NameError: name 'enb' is not defined

## Settings

In [8]:

import scapy.all as scp
import struct
import numpy as np
import matplotlib.pyplot as plt

from typing import DefaultDict

from IrisBackendv3.data_standards import DataStandards
from IrisBackendv3.data_standards.prebuilt import add_to_standards, watchdog_heartbeat_tvac
from IrisBackendv3.data_standards.logging import logger as DsLogger

from IrisBackendv3.codec.magic import Magic, MAGIC_SIZE
from IrisBackendv3.codec.settings import set_codec_standards

from IrisBackendv3.utils.basic import print_bytearray_hex as printraw

In [9]:
DsLogger.setLevel('CRITICAL')
standards = DataStandards.build_standards()
add_to_standards(standards, watchdog_heartbeat_tvac)
standards.print_overview()

Data Standards Overview: [

	Module[256]::BlockDriver

		Commands:

		Telemetry:

			0.	Channel[0]::BdCycles: uint32

		Events:

	Module[512]::RateGroupDriver

		Commands:

		Telemetry:

		Events:

	Module[768]::ActiveRateGroup-RateGroupLowFreq

		Commands:

		Telemetry:

			0.	Channel[0]::RgMaxTime: uint32

			1.	Channel[1]::RgCycleSlips: uint32

		Events:

			0.	Event[0]::RateGroupStarted[]

			1.	Event[1]::RateGroupCycleSlip[cycle: uint32]

	Module[1024]::ActiveRateGroup-RateGroupMedFreq

		Commands:

		Telemetry:

			0.	Channel[0]::RgMaxTime: uint32

			1.	Channel[1]::RgCycleSlips: uint32

		Events:

			0.	Event[0]::RateGroupStarted[]

			1.	Event[1]::RateGroupCycleSlip[cycle: uint32]

	Module[1280]::ActiveRateGroup-RateGroupHiFreq

		Commands:

		Telemetry:

			0.	Channel[0]::RgMaxTime: uint32

			1.	Channel[1]::RgCycleSlips: uint32

		Events:

			0.	Event[0]::RateGroupStarted[]

			1.	Event[1]::RateGroupCycleSlip[cycle: uint32]

	Module[1536]::CubeRoverTime

		Commands:

		Teleme

In [10]:
from IrisBackendv3.codec.fsw_data_codec import encode
bo = '<'
seq_num = 0x00
def pack(name_m: str, name_c: str, data: bytes) -> bytes:
    global seq_num
    module = standards.modules[name_m]
    cmd = module.commands[name_m+'_'+name_c]

    magic = Magic.COMMAND
    magic_bytes = magic.encode(byte_order=bo)
    opcode = struct.pack(bo+'H', module.ID | cmd.ID)

    seq_num = seq_num + 1
    vlp_len = MAGIC_SIZE + 2 + len(data)
    chk = 0x00
    CPH = struct.pack(bo+'B H B', seq_num, vlp_len, chk)

    out = CPH + magic_bytes + opcode + data
    return out
    
data = struct.pack(bo+'B H', 0x01, 0x0005) # 77 for modes, 60 for prep/deploy
packet = pack('Camera', 'TakeImage', data) 
printraw(packet)
print(packet.hex())

01 09 00 00 55 da ba 00 01 11 01 05 00
0109000055daba000111010500


In [11]:
standards.modules['WatchDogInterface'].commands.vals
sorted([m.ID for m in standards.modules.vals])

[256,
 512,
 768,
 1024,
 1280,
 1536,
 1792,
 2048,
 2304,
 2560,
 2816,
 3328,
 3584,
 3840,
 4096,
 4352,
 65280]

In [12]:
from IrisBackendv3.codec.payload import Payload, CommandPayload, WatchdogCommandPayload
from IrisBackendv3.codec.metadata import DataPathway, DataSource
from IrisBackendv3.codec.magic import Magic
from IrisBackendv3.codec.settings import ENDIANNESS_CODE, set_codec_standards

import struct

set_codec_standards(standards)

def pack_watchdog(command_name: str, **kwargs) -> bytes:
    module, command = standards.global_command_lookup(command_name)
    payload = WatchdogCommandPayload(
        pathway = DataPathway.WIRED,
        source = DataSource.GENERATED,
        magic = Magic.WATCHDOG_COMMAND,
        module_id = module.ID,
        command_id = command.ID,
        args = kwargs
    )
    return payload.encode()
packet = Magic.WATCHDOG_COMMAND.encode() + pack_watchdog('WatchDogInterface_SetVSetpoint', setpoint=1000) # camera_num=0x01, callback_id = 0x05)

vlp_len = len(packet)
checksum = 0 # ignore for now TODO: impl. me
seq_num = 0 # watchdog doesn't care
CPH = struct.pack(ENDIANNESS_CODE + 'B H B', seq_num, vlp_len, checksum)
packet = CPH + packet

import scapy.all as scp
full_packet = scp.IP(dst='127.0.0.1', src='222.173.190.239')/scp.UDP(dport=8080)/scp.Raw(load=packet)
printraw(scp.raw(scp.IP(scp.raw(full_packet))))
printraw(scp.raw(full_packet))
scp.raw(full_packet)
len(full_packet)


KeyError: 'WatchDogInterface_SetVSetpoint'

In [ ]:
# Data Transport:
file = './test-data/Iris_FSWv1.0.0_210409_Telemetry.pcapng' # PCAP logs
protocol = scp.UDP # Protocol FSW is using to send data
port = 8080 # Port on the spacecraft FSW is sending data to

# Data Formatting Settings:
packetgap = 0 # number of packets to ignore at beginning of pcap
deadspace = 0 # number of bytes of deadspace at the beginning of the
endianness_code = "<" # < = little, > = big, ! = network

# Image Formatting Settings:
image_settings = {
    "width": 2592,
    "height": 1944
}

## Decode First Packet
### Grab Packet

In [ ]:
pcap = scp.rdpcap(file)

In [ ]:
packets = list(filter(lambda x: x.dport == port, pcap[protocol][packetgap:]))
i = 0

In [ ]:
packet = packets[i]

In [ ]:
content = scp.raw(packet.getlayer(scp.Raw))[deadspace:]
printraw(content)
packet_bytes = content
# packet_bytes

00 cc 03 90 ff 10 00 c0 00 0f 00 00 00 00 04 00 ff 10 00 c0 01 0f 00 00 00 00 fd ff ff 10 00 c0 02 0f 00 00 00 00 9f 00 ff 10 00 c0 03 0f 00 00 00 00 43 00 ff 10 00 c0 04 0f 00 00 00 00 2e 00 ff 10 00 c0 05 0f 00 00 00 00 d3 ff ff 10 00 c0 00 05 00 00 00 00 00 00 00 00 ff 10 00 c0 00 04 02 00 00 00 d0 07 00 00 ff 10 00 c0 00 03 23 00 00 00 b8 88 00 00 ff 10 00 c0 00 0f f2 00 00 00 09 00 ff 10 00 c0 01 0f f2 00 00 00 fd ff ff 10 00 c0 02 0f f2 00 00 00 a0 00 ff 10 00 c0 03 0f f2 00 00 00 3b 00 ff 10 00 c0 04 0f f2 00 00 00 1d 00 ff 10 00 c0 05 0f f2 00 00 00 de ff ff 10 00 c0 00 05 24 00 00 00 e8 03 00 00 ff 10 00 c0 00 0f e5 01 00 00 04 00 ff 10 00 c0 01 0f e5 01 00 00 fd ff ff 10 00 c0 02 0f e5 01 00 00 9c 00 ff 10 00 c0 03 0f e5 01 00 00 3a 00 ff 10 00 c0 04 0f e5 01 00 00 2f 00 ff 10 00 c0 05 0f e5 01 00 00 e4 ff ff 10 00 c0 00 03 16 01 00 00 a0 8c 00 00 ff 10 00 c0 00 0f d7 02 00 00 05 00 ff 10 00 c0 01 0f d7 02 00 00 fe ff ff 10 00 c0 02 0f d7 02 00 00 a0 00 ff 10 00 c0 03 0f d7 0

### Decode Common Packet Header

In [ ]:
CPH_SIZE = 4
CPH = packet_bytes[:CPH_SIZE]
printraw(CPH)
cph_head, checksum = CPH[:-1], CPH[-1:]
seq_num, vlp_len = struct.unpack(endianness_code+' B H', cph_head)
print(seq_num, vlp_len, checksum)
#! TODO: Perform checksum check. (not impl. in FSW atm)
assert vlp_len == len(packet_bytes) - CPH_SIZE

00 cc 03 90
0 972 b'\x90'


### Decode Variable Length Payload

In [ ]:
from IrisBackendv3.codec.payload import extract_downlinked_payloads
from IrisBackendv3.codec.metadata import DataPathway, DataSource
from IrisBackendv3.codec.magic import Magic

# Extract the Variable Length Payload
VLP = packet_bytes[CPH_SIZE:]
# printraw(VLP)


# Parse VLP:
payloads = extract_downlinked_payloads(
    VLP = VLP,
    pathway = DataPathway.WIRELESS,
    source = DataSource.PCAP
)

In [ ]:
sorted(standards.modules.vals, key = lambda m: m.ID)

[Module[256]::BlockDriver,
 Module[512]::RateGroupDriver,
 Module[768]::ActiveRateGroup-RateGroupLowFreq,
 Module[1024]::ActiveRateGroup-RateGroupMedFreq,
 Module[1280]::ActiveRateGroup-RateGroupHiFreq,
 Module[1536]::CubeRoverTime,
 Module[1792]::TlmChan,
 Module[2048]::CommandDispatcher,
 Module[2304]::GroundInterface,
 Module[2560]::NetworkManager,
 Module[2816]::ActiveLogger,
 Module[3328]::Navigation,
 Module[3584]::MotorControl,
 Module[3840]::Imu,
 Module[4096]::WatchDogInterface,
 Module[4352]::Camera,
 Module[65280]::WatchdogHeartbeatTvac]

In [ ]:
if magic == Magic.TELEMETRY:
    # Grab Telemetry Header:
    module_id, channel_id, data_size, VLP = VLP[:1], VLP[1:2], VLP[2:4], VLP[4:]
    print(module_id, channel_id, data_size)
    # Unpack + Format Telemetry Header:
    module_id = struct.unpack(endianness_code+'B', module_id)[0] << 8
    channel_id = struct.unpack(endianness_code+'B', channel_id)[0]
    #! TODO: Not impl. in fsw. Once is, perform check against expected size.
    # ... Rn, replace w/expected size
    data_size = struct.unpack(endianness_code+'H', data_size)[0]
    try:
        module = standards.modules[module_id]
    except KeyError:
        pass # logger
    exp_data_size = standards.modules[module_id] 

    print(hex(module_id), hex(channel_id), data_size)
    # data, timestamp, VLP = VLP[:data_size], VLP[data_size:data_size+4], VLP[data_size+4:]

SyntaxError: invalid syntax (<ipython-input-18-1472e03e6c7f>, line 14)

In [ ]:
standards.modules['Imu'].telemetry

[((['XAcc'], 0), Channel[0]::XAcc: int16), ((['YAcc'], 1), Channel[1]::YAcc: int16), ((['ZAcc'], 2), Channel[2]::ZAcc: int16), ((['XAng'], 3), Channel[3]::XAng: uint16), ((['YAng'], 4), Channel[4]::YAng: uint16), ((['ZAng'], 5), Channel[5]::ZAng: uint16)]

In [ ]:
[t.datatype for t in standards.modules['Imu'].telemetry.vals]

[<FswDataType.I16: 'int16'>,
 <FswDataType.I16: 'int16'>,
 <FswDataType.I16: 'int16'>,
 <FswDataType.U16: 'uint16'>,
 <FswDataType.U16: 'uint16'>,
 <FswDataType.U16: 'uint16'>]

In [ ]:
from __future__ import annotations
from typing import Optional, Callable, Dict, List
import struct
import pickle
from abc import abstractmethod
from enum import Enum

class Dummy:
    __slots__: List[str] = [
        '_raw',
        '_endianness_code'
    ]

    class FileTypeMagic(Enum):
        """
        Enumeration of all file types which could be downlinked.

        As usual, for backwards compatibility and data preservation, *never* change 
        any of the enum values or delete entries, just deprecate old ones.
        """
        IMAGE = 0x01
        UWB = 0x0F

    FTM: FileTypeMagic = FileTypeMagic.UWB

    _raw: Optional[bytes]
    _endianness_code: str

    def __init__(self,
        raw: Optional[bytes] = None,
        endianness_code = '!'
    ) -> None:
        self._raw = raw
        self._endianness_code = endianness_code

    @abstractmethod
    def encode(self) -> bytes:
        raise NotImplementedError()

    @classmethod
    @abstractmethod
    def decode(cls, raw: bytes, endianness_code:str) -> Dummy:
        raise NotImplementedError()

    def __reduce__(self) -> Tuple[Callable, Tuple[bytes, str], Optional[str]]:
        print("Dummy Reduce")

        if self._raw is None:
            self._raw = self.encode()

        if hasattr(self, '__getstate__'):
            state = self.__getattribute__('__getstate__')()
        else:
            state = None

        # "Callable object" returned will be the decoding function:
        # If a subclassed object is reduced, it will call that subclass' `decode`
        # function (assuming it's been implemented).
        return (self.__class__.decode, (self._raw, self._endianness_code), state)

class DummerInterface(Dummy):
    __slots__: List[str] = [
        'other_thing',
        '_data'
    ]

    other_thing: str
    _data: int

class DummyThicc(DummerInterface):
    __slots__: List[str] = []

    # FTM2: DummyThicc.FileTypeMagic = DummyThicc.FileTypeMagic.IMAGE

    def __init__(self,
        data: int,
        other_thing: str = "",
        raw: Optional[bytes] = None,
        endianness_code = '!'
    ) -> None:
        self.other_thing = other_thing
        self._data = data
        super().__init__(raw, endianness_code)

    def encode(self) -> bytes:
        print("Encoding")
        return struct.pack(self._endianness_code+'L', self._data)

    @classmethod
    def decode(cls, raw: bytes, endianness_code:str) -> Dummy:
        print("Decoding")
        return cls(
            data = struct.unpack(endianness_code+'L', raw)[0],
            raw = raw,
            endianness_code = endianness_code
        )

    def __repr__(self) -> str:
        return f"{self.other_thing}[{self._data}] -> {self._endianness_code}{self._raw}"

    def __eq__(self, other) -> bool:
        return self.other_thing == other.other_thing and self._data == other._data

    def __getstate__(self) -> str:
        print("Getting State")
        return self.other_thing

    def __setstate__(self, state: str) -> None:
        print("Setting State")
        self.other_thing = state


A = DummyThicc(other_thing='Apple', data=5)
B = DummyThicc(other_thing='Dead', data=0xBEEF)
print((A, B))

print('--pickle--')
pA = pickle.dumps(A)
pB = pickle.dumps(B)
print('--unpickle--')
upA = pickle.loads(pA)
upB = pickle.loads(pB)

print((A, B))
print((upA, upB))
print((upA==A, upB==B))

from typing import NamedTuple

class X(NamedTuple):
    a: int
    b: str

x = X(
    a = 0,
    b = '5'
)
getattr(x, 'b')
# Dummy.FTM == DummyThicc.FileTypeMagic.UWB, DummyThicc.FTM2
DummerInterface.__slots__


(Apple[5] -> !None, Dead[48879] -> !None)
--pickle--
Dummy Reduce
Encoding
Getting State
Dummy Reduce
Encoding
Getting State
--unpickle--
Decoding
Setting State
Decoding
Setting State
(Apple[5] -> !b'\x00\x00\x00\x05', Dead[48879] -> !b'\x00\x00\xbe\xef')
(Apple[5] -> !b'\x00\x00\x00\x05', Dead[48879] -> !b'\x00\x00\xbe\xef')
(True, True)


['other_thing', '_data']

In [ ]:
from typing import List

def cdec(module_name):
    def dec_inner(targ_class):
        targ_class.MODULE_NAME: str = module_name # type: ignore

        @property
        def tprop(self):
            return self.a + 100
        targ_class.tprop = tprop

        def inner_init(self, a: int):
            self.a = a
        targ_class.__init__ = inner_init
        
        return targ_class
    return dec_inner

def subcdec(module_name):
    def dec_inner(targ_class):
        # targ_class = cdec(module_name)(targ_class)

        def sub_inner_init(self, b: int):
            self.b = b
            type(self).mro()[1].__init__(self, a=b//2)
        targ_class.__init__ = sub_inner_init
        
        return targ_class
    return dec_inner

@cdec('mod')
class cex:
    __slots__: List[str] = ['a']
    TAG: str='CEX'

    @classmethod
    def get_cls_tag(cls):
        return cls.TAG

    @staticmethod
    def double(x):
        return x*2

@subcdec('modd')
class subcex(cex):
    __slots__: List[str] = ['b']
    TAG: str='SUBCEX'

A = cex(55)
B = subcex(66)

print((A.get_cls_tag(), B.get_cls_tag()))
print(((A.a,), A.tprop, A.MODULE_NAME, id(A.MODULE_NAME), cex.MODULE_NAME, id(cex.MODULE_NAME)))
print(((B.a, B.b), B.tprop, B.MODULE_NAME, id(B.MODULE_NAME), cex.MODULE_NAME, id(cex.MODULE_NAME)))
cex.MODULE_NAME = 'x'
print((A.a, A.tprop, A.MODULE_NAME, id(A.MODULE_NAME), cex.MODULE_NAME, id(cex.MODULE_NAME)))
print((B.a, B.tprop, B.MODULE_NAME, id(B.MODULE_NAME), cex.MODULE_NAME, id(cex.MODULE_NAME)))
print((A.__slots__,B.__slots__))

8
('CEX', 'SUBCEX')
((55,), 155, 'mod', 4379173360, 'mod', 4379173360)
((33, 66), 133, 'mod', 4379173360, 'mod', 4379173360)
(55, 155, 'x', 4378863536, 'x', 4378863536)
(33, 133, 'x', 4378863536, 'x', 4378863536)
(['a'], ['b'])
